In [1]:
import os
import sys

from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import colors
import pandas as pd
import IPython.display as ipd
import glob
import numpy as np
# import muspy
# import pypianoroll
import torch
# from util.play_midi import play_midi

import miditok
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
from symusic import Score
from torch.utils.data import DataLoader
from torch.cuda import is_available as cuda_available, is_bf16_supported

from transformers import AutoModelForCausalLM, MistralConfig, Trainer, TrainingArguments, GenerationConfig, BitsAndBytesConfig
from tqdm import tqdm
from evaluate import load as load_metric

In [2]:
midi_files = list(Path("/user/HS400/ye00098/Downloads/pretraining_dataset_2/").glob("*.mid"))

tokenizer = miditok.REMI.from_pretrained("theglassofwater/remi_12500")

print(len(midi_files))


25000


/user/HS400/ye00098/.local/lib/python3.10/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/user/HS400/ye00098/.local/lib/python3.10/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


In [9]:
# seq_len = i8192*.75
seq_len = 4096
model_config = MistralConfig(
    vocab_size=len(tokenizer),
    hidden_size=512,
    intermediate_size=2048,
    num_hidden_layers=12,
    num_attention_heads=8,
    num_key_value_heads=4,
    sliding_window=512,
    max_position_embeddings=seq_len,
    pad_token_id=tokenizer["PAD_None"],
    bos_token_ids=tokenizer["BOS_None"],
    eos_token_ids=tokenizer["EOS_None"]
    )

# quantization_config = BitsAndBytesConfig( # when loading a model for inference only
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.float16,
# )

model = AutoModelForCausalLM.from_config(model_config) #, torch_dtype = torch.float16, attn_implementation = "flash_attention_2") # , quantization_config=True)
param_count = sum(i.numel() for i in model.parameters())
print(f"Parameter count: {param_count} = {param_count/1e6:.0f}M" )
model

Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0
}



Parameter count: 59998720 = 60M


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(12500, 512, padding_idx=0)
    (layers): ModuleList(
      (0-11): 12 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=256, bias=False)
          (v_proj): Linear(in_features=512, out_features=256, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=512, out_features=2048, bias=False)
          (up_proj): Linear(in_features=512, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [10]:
from transformers import DataCollatorForLanguageModeling


train_dataset = DatasetMIDI(
    files_paths=midi_files[:int(len(midi_files)*0.7)],
    max_seq_len=seq_len+1,  # max_seq_len = start + seq_len + end
    tokenizer=tokenizer,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

eval_dataset = DatasetMIDI(
    files_paths=midi_files[int(len(midi_files)*0.7):],
    max_seq_len=seq_len,  # max_seq_len = start + seq_len + end
    tokenizer=tokenizer,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

collator = DataCollator(
    pad_token_id=tokenizer["PAD_None"],
    copy_inputs_as_labels=True,

)
# DataCollator()

# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

data_loader = DataLoader(dataset=train_dataset, batch_size=16, collate_fn=collator,)

In [ ]:
# seq_lengths_train = []
# for i in train_dataset:
#     seq_lengths_train.append(len(i["input_ids"]))
# print(f"mean song lengt

In [11]:
# USE_CUDA = cuda_available()
# if not cuda_available():
#     FP16 = FP16_EVAL = BF16 = BF16_EVAL = False
# elif is_bf16_supported():
#     BF16 = BF16_EVAL = True
#     FP16 = FP16_EVAL = False
# else:
#     BF16 = BF16_EVAL = False
#     FP16 = FP16_EVAL = True
# USE_MPS = not USE_CUDA and mps_available()

metrics = {metric: load_metric(metric) for metric in ["accuracy"]}
def compute_metrics(eval_pred):
    """
    Compute metrics for pretraining.

    Must use preprocess_logits function that converts logits to predictions (argmax or sampling).

    :param eval_pred: EvalPrediction containing predictions and labels
    :return: metrics
    """
    predictions, labels = eval_pred
    not_pad_mask = labels != -100
    labels, predictions = labels[not_pad_mask], predictions[not_pad_mask]
    return metrics["accuracy"].compute(predictions=predictions.flatten(), references=labels.flatten())

def preprocess_logits(logits):
    """
    Preprocess the logits before accumulating them during evaluation.

    This allows to significantly reduce the memory usage and make the training tractable.
    """
    pred_ids = torch.argmax(logits, dim=-1)  # long dtype
    return pred_ids
    
# Create config for the Trainer
USE_CUDA = cuda_available()
if not cuda_available():
    FP16 = FP16_EVAL = BF16 = BF16_EVAL = False
elif is_bf16_supported():
    BF16 = BF16_EVAL = True
    FP16 = FP16_EVAL = False
else:
    BF16 = BF16_EVAL = False
    FP16 = FP16_EVAL = True

training_config = TrainingArguments(
    output_dir = "hello",
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=3,
    eval_accumulation_steps=None,
    eval_steps=200,
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=3.0,
    max_steps=200,
    # num_train_epochs=5,
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.3,
    log_level="debug",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=5,
    # no_cuda=not USE_CUDA,
    # seed=444,
    # fp16=FP16,
    # fp16_full_eval=FP16_EVAL,
    # bf16=BF16,
    # bf16_full_eval=BF16_EVAL,
    load_best_model_at_end=True,
    label_smoothing_factor=0.,
    optim="adamw_torch",
    # report_to=["tensorboard"],
    gradient_checkpointing=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [12]:
trainer = Trainer(
    model=model,
    args=training_config,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=None,
    preprocess_logits_for_metrics=preprocess_logits,
)

# Training
train_result = trainer.train()

max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 6
***** Running training *****
  Num examples = 17,500
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 18
  Gradient Accumulation steps = 3
  Total optimization steps = 200
  Number of trainable parameters = 59,998,720


  0%|          | 0/200 [00:00<?, ?it/s]

/user/HS400/ye00098/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.14 GiB. GPU 0 has a total capacity of 7.92 GiB of which 809.00 MiB is free. Including non-PyTorch memory, this process has 7.12 GiB memory in use. Of the allocated memory 6.19 GiB is allocated by PyTorch, and 848.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)